In [1]:
#Funcation_Bayesian_Classification
import numpy as np
import math as m
from pyecharts import Style
from pyecharts import Scatter
from pyecharts import EffectScatter
from pyecharts import Overlap

#定义贝叶斯分类模型
class Bayesian_Classification():
    
    #初始化
    def __init__(self):
   
        #self.avr_w1          类1的均值向量 
        #self.avr_w2          类2的均值向量
        #self.cov_w1          类1的协方差
        #self.cov_w2          类2的协方差
        #self.correct_w1      类1的正确分类的数据
        #self.correct_w2      类2的正确分类的数据
        #self.error_w1        类1的错误分类的数据
        #self.error_w2        类2的错误分类的数据
    
        self.avr_w1=[]
        self.avr_w2=[]
        self.cov_w1=[]
        self.cov_w2=[]
        self.correct_w1=[0,0]
        self.correct_w2=[0,0]
        self.error_w1=[0,0]
        self.error_w2=[0,0]
       
    
    #贝叶斯训练
    def Bayesian_Train(self,W1,W2):

        #计算两类训练数据的均值向量
        self.avr_w1=np.mean(W1,axis=0)  #按列求均值
        self.avr_w2=np.mean(W2,axis=0)  #按列求均值
        
        #计算w1类的协方差矩阵
        dim0_w1=W1[:,0]
        dim1_w1=W1[:,1]

        var_01_w1=sum((dim0_w1 - self.avr_w1[0]) * (dim1_w1 - self.avr_w1[1])) / (len(W1) -1)
        avr_dim0_w1=np.var(dim0_w1)
        avr_dim1_w1=np.var(dim1_w1)

        cov_w1=[[avr_dim0_w1, var_01_w1],[var_01_w1, avr_dim1_w1]]

        #计算w2类的协方差矩阵
        dim0_w2=W2[:,0]
        dim1_w2=W2[:,1]

        var_01_w2=sum((dim0_w2 - self.avr_w2[0]) * (dim1_w2 - self.avr_w2[1])) / (len(W2) - 1)

        avr_dim0_w2=np.var(dim0_w2)
        avr_dim1_w2=np.var(dim1_w2)

        cov_w2=[[avr_dim0_w2, var_01_w2],[var_01_w2 ,avr_dim1_w2 ]]

        #转换为矩阵形式
        self.cov_w1=np.mat(cov_w1)
        self.cov_w2=np.mat(cov_w2)
           
    def Bayesian_Classify(self,W1,W2):
        #先验概率
        P_w1=1/2;
        P_w2=1/2;
        
        Num=len(W1)
        
        #合并数据集
        Data_Test=np.vstack((W1,W2))
       
        self.avr_w1=np.mat(self.avr_w1)
        self.avr_w2=np.mat(self.avr_w2)
        data_test=np.mat(Data_Test)

       
        for i in range(0,Num*2):
            #导入测试数据
            x=Data_Test[i,:]
            #计算测试向量的判决函数
            J1=float(-0.5*(x-self.avr_w1)*self.cov_w1.I*(x-self.avr_w1).T+m.log(P_w1)-0.5*m.log(np.linalg.det(self.cov_w1)))
            J2=float(-0.5*(x-self.avr_w2)*self.cov_w2.I*(x-self.avr_w2).T+m.log(P_w2)-0.5*m.log(np.linalg.det(self.cov_w2)))
           
            #对测试集进行分类
            #类w1测试样本归为类w1
            if i<Num and J1 >= J2:
                self.correct_w1=np.vstack((self.correct_w1,Data_Test[i,:]))
            #类w2测试样本归为类w2
            elif i>=Num and J1 < J2:
                self.correct_w2=np.vstack((self.correct_w2,Data_Test[i,:]))
            #类w1测试样本误判为类w2
            elif (i<Num and J1 < J2):
                self.error_w1=np.vstack((self.error_w1,Data_Test[i,:]))
            #类w2测试样本误判为类w1
            elif (i>=Num and J1 >= J2):
                self.error_w2=np.vstack((self.error_w2,Data_Test[i,:]))

        ##计算分类的正确率与错误率
        Num_correct_w1=len(self.correct_w1)-1
        Num_correct_w2=len(self.correct_w2)-1 
        Num_error_w1=len(self.error_w1)-1
        Num_error_w2=len(self.error_w2)-1

        error_rate_w1=Num_error_w1/Num
        error_rate_w2=Num_error_w2/Num
        correct_rate_w1=1-error_rate_w1
        correct_rate_w2=1-error_rate_w2

        print('w1正确率: %2.2f%%'%(correct_rate_w1*100))
        print('w2正确率: %2.2f%%'%(correct_rate_w2*100))
        print('w1误分率: %2.2f%%'%(error_rate_w1*100))
        print('w2误分率: %2.2f%%'%(error_rate_w2*100))
        
    #pyecharts 分类结果的可视化
   
    def Show(self,Title):
        
        s_1 = Scatter(title=Title,title_text_size=20, title_pos="center",width=1000,height=500)
        s_1.add("w1",self.correct_w1[:,0], self.correct_w1[:,1],symbol_size=6,color='#FF4500',
              xaxis_max=12,
              xaxis_min=17,
              xaxis_name='光伏阵列输出电压（V）',
              yaxis_name='光伏阵列输出电流（A）',
              yaxis_name_gap=45,
              legend_pos='center',
              legend_top="6%",
              legend_text_size=14)
        
        s_2 = Scatter(title=Title,title_text_size=20, title_pos="center",width=1000,height=500)
        s_2.add("w2",self.correct_w2[:,0], self.correct_w2[:,1],symbol_size=6,xaxis_max=12,
              xaxis_min=17,
              xaxis_name='光伏阵列输出电压（V）',
              yaxis_name='光伏阵列输出电流（A）',
              yaxis_name_gap=45,
              legend_pos='center',
              legend_top="6%",
              legend_text_size=14)
        
        es_1 = EffectScatter(title=Title,title_text_size=20, title_pos="center",width=1000,height=500)
        es_1.add("w21",self.error_w1[:,0], self.error_w1[:,1],symbol_size=6,xaxis_max=12,color='#FF4500',
              xaxis_min=17,
              xaxis_name='光伏阵列输出电压（V）',
              yaxis_name='光伏阵列输出电流（A）',
              yaxis_name_gap=45,
              legend_pos='center',
              legend_top="6%",
              legend_text_size=14)
        
        es_2 = EffectScatter(title=Title,title_text_size=20, title_pos="center",width=1000,height=500)
        es_2.add("w12",self.error_w2[:,0], self.error_w2[:,1],
              symbol_size=6,
              xaxis_max=12,
              xaxis_min=17,
              xaxis_name='光伏阵列输出电压（V）',
              yaxis_name='光伏阵列输出电流（A）',
              yaxis_name_gap=45,
              legend_pos='center',
              legend_top="6%",
              legend_text_size=14)
        
        #s.use_theme("roma")
        #es.use_theme("roma")

       
        overlap = Overlap()
        overlap.add(s_1)
        overlap.add(es_1)
        overlap.add(s_2)
       
        overlap.add(es_2)
        
        overlap.render()
    
    #两类训练数据的均值向量
    def get_avr_w1(self):
        return self.avr_w1
    
    def get_avr_w2(self):
        return self.avr_w2
    #两类训练数据的协方差
    def get_cov_w1(self):
        return self.cov_w1
    
    def get_cov_w2(self):
        return self.cov_w2
    
    def get_correct_w1(self):
        return self.correct_w1
    
    def get_correct_w2(self):
        return self.correct_w2
    
    def get_error_w1(self):
        return self.error_w1
    
    def get_error_w2(self):
        return self.error_w2
        


In [2]:
#################################################################################

#导入训练
Training_Data=np.loadtxt('D:/杭电与人工智能研究院/光伏教仪/人工智能课程案例/课程实验课件/机器学习课程实验一___贝叶斯估计/课程实\
验程序/数据/Training_Data.txt')
    
# Training_Data 800*3  400组正常运行数据  400组阴影遮挡数据 

#80%训练数据与20%测试数据
M,N=Training_Data.shape

#在各组中随机地抽取Num_train个训练样本
Num_train=int(M//2*0.8)
temp_w1=np.random.permutation(M//2)
temp_w2=400+np.random.permutation(M//2)

#训练集w1和w2各Num_train个样本
Data_Train_W1=Training_Data[temp_w1[0:Num_train],0:2]
Data_Train_W2=Training_Data[temp_w2[0:Num_train],0:2]

#测试集w1和w2各m//2-Num_train个样本
Data_Test_W1=Training_Data[temp_w1[Num_train:M//2],0:2]
Data_Test_W2=Training_Data[temp_w2[Num_train:M//2],0:2]

#创建一个实例
bayesian_classification=Bayesian_Classification()

#训练模型的分类结果
bayesian_classification.Bayesian_Train(Data_Train_W1,Data_Train_W2)

#测试模型的分类结果
bayesian_classification.Bayesian_Classify(Data_Test_W1,Data_Test_W2)

bayesian_classification.Show("贝叶斯分类-测试集")

w1正确率: 72.50%
w2正确率: 76.25%
w1误分率: 27.50%
w2误分率: 23.75%
